Notes:
* The data is for two years: 2023 and 2024
* The issuer_bank_name has only value of "My Favorite Bank". Possible to drop
* Null values seem to be reasonable.

In [2]:
import polars as pl

data = pl.read_parquet("data/DECENTRATHON_3.0.parquet")

In [8]:
data.head()

transaction_id,transaction_timestamp,card_id,expiry_date,issuer_bank_name,merchant_id,merchant_mcc,mcc_category,merchant_city,transaction_type,transaction_amount_kzt,original_amount,transaction_currency,acquirer_country_iso,pos_entry_mode,wallet_type,__index_level_0__
str,datetime[ns],i64,str,str,i64,i64,str,str,str,f64,f64,str,str,str,str,i64
"""ffb21bbd-fef1-4983-9774-3b01c8…",2023-08-12 23:14:19.043176,10000,"""09/26""","""My Favorite Bank""",50359,5499,"""Grocery & Food Markets""","""Shymkent""","""POS""",1237.66,null,"""KZT""","""KAZ""","""Contactless""","""Apple Pay""",0
"""acd754f9-bd4c-416d-babc-926050…",2023-08-12 17:07:08.641030,10000,"""09/26""","""My Favorite Bank""",50359,5499,"""Grocery & Food Markets""","""Shymkent""","""POS""",2328.03,null,"""KZT""","""KAZ""","""QR_Code""","""Bank's QR""",1
"""b023697b-eb35-40e6-af75-3586c6…",2023-08-12 08:45:32.946846,10000,"""09/26""","""My Favorite Bank""",50214,4814,"""Utilities & Bill Payments""","""Almaty""","""BILL_PAYMENT""",53871.78,null,"""KZT""","""KAZ""",null,null,2
"""d59d8537-1e48-41b5-91e1-dc7cf9…",2023-08-12 10:20:20.242510,10000,"""09/26""","""My Favorite Bank""",50279,8011,"""Pharmacies & Health""","""Shymkent""","""POS""",22501.56,null,"""KZT""","""KAZ""","""Contactless""","""Apple Pay""",3
"""85d2eef6-3b09-47a6-987f-f45ab4…",2023-08-12 00:42:00.556252,10000,"""09/26""","""My Favorite Bank""",50332,5499,"""Grocery & Food Markets""","""Almaty""","""POS""",1224.04,null,"""KZT""","""KAZ""","""QR_Code""","""Bank's QR""",4


In [12]:
data.get_column("mcc_category").unique()

mcc_category
str
"""Services (Other)"""
"""Unknown"""
"""Clothing & Apparel"""
"""Electronics & Software"""
"""Travel & Transportation"""
…
"""Pharmacies & Health"""
"""General Retail & Department"""
"""Hobby, Books, Sporting Goods"""


In [3]:
data.null_count()

transaction_id,transaction_timestamp,card_id,expiry_date,issuer_bank_name,merchant_id,merchant_mcc,mcc_category,merchant_city,transaction_type,transaction_amount_kzt,original_amount,transaction_currency,acquirer_country_iso,pos_entry_mode,wallet_type,__index_level_0__
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,2470036,2470036,0,2469297,0,0,11531433,0,0,2653412,4673494,0


In [4]:
data.describe()

statistic,transaction_id,transaction_timestamp,card_id,expiry_date,issuer_bank_name,merchant_id,merchant_mcc,mcc_category,merchant_city,transaction_type,transaction_amount_kzt,original_amount,transaction_currency,acquirer_country_iso,pos_entry_mode,wallet_type,__index_level_0__
str,str,str,f64,str,str,f64,f64,str,str,str,f64,f64,str,str,str,str,f64
"""count""","""11536850""","""11536850""",1.153685e7,"""11536850""","""11536850""",9.066814e6,9.066814e6,"""11536850""","""9067553""","""11536850""",1.153685e7,5417.0,"""11536850""","""11536850""","""8883438""","""6863356""",1.153685e7
"""null_count""","""0""","""0""",0.0,"""0""","""0""",2.470036e6,2.470036e6,"""0""","""2469297""","""0""",0.0,1.1531433e7,"""0""","""0""","""2653412""","""4673494""",0.0
"""mean""",null,"""2024-04-23 10:50:42.587606""",11003.031321,null,null,50249.453106,5644.688338,null,null,null,20166.686882,74316.000809,null,null,null,null,5768424.5
"""std""",null,null,580.572998,null,null,143.985954,1030.229349,null,null,null,81305.720721,575082.106378,null,null,null,null,3.3304e6
"""min""","""0000013f-41fa-4e8e-b029-df002c…","""2023-01-01 00:30:12.551745""",10000.0,"""01/25""","""My Favorite Bank""",50000.0,3000.0,"""Clothing & Apparel""","""Aktobe""","""ATM_WITHDRAWAL""",100.0,0.21,"""AED""","""ARE""","""Chip""","""Apple Pay""",0.0
"""25%""",null,"""2023-12-21 14:19:07.908075""",10494.0,null,null,50126.0,5200.0,null,null,null,2140.45,28.8,null,null,null,null,2.884212e6
"""50%""",null,"""2024-05-19 05:29:53.788712""",10998.0,null,null,50248.0,5499.0,null,null,null,6070.66,164.78,null,null,null,null,5.768425e6
"""75%""",null,"""2024-09-17 17:55:15.468680""",11504.0,null,null,50375.0,5812.0,null,null,null,19259.91,2086.07,null,null,null,null,8.652637e6
"""max""","""fffffe8e-4a4f-4ba7-990c-19c995…","""2024-12-31 23:59:35.437993""",11999.0,"""12/29""","""My Favorite Bank""",50499.0,9311.0,"""Utilities & Bill Payments""","""Shymkent""","""SALARY""",3.4073e7,2.7190e7,"""UZS""","""UZB""","""Swipe""","""Samsung Pay""",1.1536849e7
